In [1]:
import sys
import argparse
import os

sys.argv = ["view", "--config", "../../config/single_task_object_detection.yaml"]

parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
args = parser.parse_args()

print(args.config)

../../config/single_task_object_detection.yaml


In [2]:
from dataloader import VOC08Attr
from torchvision.transforms import transforms
from config_experiments import config
from bbox_transform import relative_to_absolute_bbox
from model import ObjectDetectionModel
import torch
import numpy as np
import torch.onnx
import netron

In [3]:
os.getcwd()
os.chdir("../../")
os.getcwd()

'/data01/dl23vitcas/dl_project'

In [4]:
transform_train = transforms.Compose(
    [
        transforms.Resize(size=config["transform"]["resize_values"]),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=config["transform"]["mean"], std=config["transform"]["std"]
        ),
    ]
)

In [5]:
model = ObjectDetectionModel()
model.eval()

data_train = VOC08Attr(transform=transform_train, train=True)

image, gt_class, gt_bbox, gt_attributes, ss_rois = data_train[0]

num_roi = ss_rois.shape[0]
ridx = np.zeros(num_roi).astype(int)
ridx = torch.from_numpy(ridx).unsqueeze(-1)

name_file = "model_obj.onnx"

torch.onnx.export(model, (image.unsqueeze(0), ss_rois, ridx), name_file)
netron.start(name_file)

/home/dl23vitcas/anaconda3/envs/dl/lib/python3.11/site-packages/torch/__init__.py:1209: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Serving 'model_obj.onnx' at http://localhost:8080


('localhost', 8080)